# Extract sequence by chromosome 

In [40]:
def extract_chromosomes(fasta_file):
    with open(fasta_file, 'r') as file:
        chromosome_name = None
        chromosome_seq = []
        for line in file:
            if line.startswith('>'):
                if chromosome_name is not None:
                    print(chromosome_name)
                    # Write the previous chromosome to a file
                    with open(f"../genomic_data/t2t_genome/{chromosome_name}.txt", 'w') as output_file:
                        output_file.write(''.join(chromosome_seq))
                
                # Start a new chromosome
                chromosome_name = line[1:].strip().split()[0]
                chromosome_seq = []
            else:
                chromosome_seq.append(line.strip())

        # Write the last chromosome to a file
        if chromosome_name is not None:
            with open(f"{chromosome_name}.txt", 'w') as output_file:
                output_file.write(''.join(chromosome_seq))

# Example usage
fasta_file = '../genomic_data/t2t_genome/hs1.fa'
extract_chromosomes(fasta_file)


chr1
chr2
chr3
chr4
chr5
chr6
chr7
chrX
chr9
chr8
chr11
chr10
chr12
chr13
chr14
chr15
chr16
chr17
chr18
chr20
chrY
chr19
chr22
chr21


In [7]:
import os

def calculate_percentage_of_genome(output_dir):
    chromosome_lengths = {}
    total_length = 0

    # Iterate over each file in the output directory
    for filename in os.listdir(output_dir):
        if filename.endswith(".txt") and filename.startswith("chr"):
            #print(filename)
            filepath = os.path.join(output_dir, filename)
            with open(filepath, 'r') as file:
                content = file.read()
                chromosome_length = len(content)
                chromosome_name = os.path.splitext(filename)[0]
                chromosome_lengths[chromosome_name] = chromosome_length
                total_length += chromosome_length

    # Calculate and print the percentage of each chromosome
    for chromosome, length in chromosome_lengths.items():
        percentage = (length / total_length) * 100
        print(f"Chromosome {chromosome}: {percentage:.2f}% of the total genome")

# Example usage
output_dir = '../genomic_data/t2t_genome'
calculate_percentage_of_genome(output_dir)


Chromosome chr1: 7.97% of the total genome
Chromosome chr10: 4.32% of the total genome
Chromosome chr18: 2.58% of the total genome
Chromosome chr14: 3.25% of the total genome
Chromosome chrX: 4.95% of the total genome
Chromosome chr4: 6.21% of the total genome
Chromosome chrY: 2.00% of the total genome
Chromosome chr11: 4.33% of the total genome
Chromosome chr6: 5.52% of the total genome
Chromosome chr20: 2.12% of the total genome
Chromosome chr3: 6.45% of the total genome
Chromosome chr7: 5.15% of the total genome
Chromosome chr13: 3.64% of the total genome
Chromosome chr9: 4.83% of the total genome
Chromosome chr2: 7.79% of the total genome
Chromosome chr17: 2.70% of the total genome
Chromosome chr19: 1.98% of the total genome
Chromosome chr21: 1.45% of the total genome
Chromosome chr22: 1.65% of the total genome
Chromosome chr15: 3.20% of the total genome
Chromosome chr16: 3.09% of the total genome
Chromosome chr12: 4.28% of the total genome
Chromosome chr5: 5.84% of the total genom

In [3]:
#X,Y,21,22
2+4.95+1.65+1.45

10.049999999999999

In [5]:
#20,19,18,17
2.12+1.98+2.58+2.70

9.379999999999999

# Annotation

In [5]:
import pandas as pd

#### censat

In [22]:
censat_path = '../genomic_data/t2t_annotation/chm13v2.0_censat_v2.1.bed'
censats = pd.read_csv(censat_path, sep='\t', header=None)

In [23]:
censats_chr22 = censats[censats[0] == 'chr22']
censats_chr22.head(5)

,0,1,2,3,4,5,6,7,8
1943,chr22,0,4578,censat_22_1(TAR),100,.,0,4578,"0,204,204"
1944,chr22,4578,67032,ct_22_1,100,.,4578,67032,"224,224,224"
1945,chr22,67032,226744,censat_22_2(ACRO_composite),100,.,67032,226744,"0,204,204"
1946,chr22,226744,226952,ct_22_2,100,.,226744,226952,"224,224,224"
1947,chr22,226952,230153,gsat_22_1(GSATII),100,.,226952,230153,"172,51,199"


In [24]:
def extract_general_class(value):
    if '_' in value:
        parts = value.split('_')
        general_class = parts[0]
        return general_class
    return value

for idx, row in censats_chr22.iterrows():
    assert(row[1] == row[6])
    assert(row[2] == row[7])
    assert(row[5]=='.')
    assert(row[4]==100)

censats_chr22 = censats_chr22.drop([0,4,5,6,7,8], axis=1).reset_index(drop=True)
censats_chr22.columns=['start','end', 'type']
censats_chr22['length']=censats_chr22['end']-censats_chr22['start']
censats_chr22['class'] = censats_chr22['type'].apply(extract_general_class)
# filter out transition regions
censats_chr22= censats_chr22[censats_chr22['class']!='ct']
print(censats_chr22['class'].unique())
censats_chr22.to_csv('../genomic_data/t2t_annotation/chr22/censats_chr22.csv', index=False)
censats_chr22.head(5)

['censat' 'gsat' 'hsat3' 'bsat' 'hsat1A' 'rDNA' 'hsat1B' 'hor' 'mon'
 'hsat2' 'dhor']


,start,end,type,length,class
0,0,4578,censat_22_1(TAR),4578,censat
2,67032,226744,censat_22_2(ACRO_composite),159712,censat
4,226952,230153,gsat_22_1(GSATII),3201,gsat
6,248029,251879,censat_22_3(SATR),3850,censat
8,277758,349847,censat_22_4(NovelTandem),72089,censat


#### telomeres

In [119]:
telomere_path= '../genomic_data/t2t_annotation/chm13v2.0_telomere.bed'
telomeres = pd.read_csv(telomere_path, sep='\t', header=None)

In [168]:
chr22_tel = telomeres[telomeres[0] == 'chr22'].reset_index(drop=True)
chr22_tel = chr22_tel.drop([0], axis=1)
chr22_tel.columns=['start','end']
chr22_tel.to_csv('../genomic_data/t2t_annotation/chr22/chr22_tel.csv', index=False)
chr22_tel

,start,end
0,0,4600
1,51321800,51324926


####   composite-repeats

In [25]:
comp_repeats_path='../genomic_data/t2t_annotation/chm13v2.0_composite-repeats_2022DEC.bed'
comp_repeats=pd.read_csv(comp_repeats_path, sep='\t', header=None)
comp_repeats.head(5)

,0,1,2,3
0,chr1,128098614,128594785,LSAU-BSAT
1,chr3,75720995,75725254,LSAU-BSAT
2,chr3,95366301,95392308,LSAU-BSAT
3,chr4,193388217,193550041,LSAU-BSAT
4,chr5,49999057,50026873,LSAU-BSAT


In [167]:
comp_repeats_chr22=comp_repeats[comp_repeats[0]=='chr22']
comp_repeats_chr22 = comp_repeats_chr22.drop([0,3], axis=1).reset_index(drop=True)
comp_repeats_chr22.columns=['start','end']
comp_repeats_chr22.to_csv('../genomic_data/t2t_annotation/chr22/comp_repeats_chr22.csv', index=False)
comp_repeats_chr22.head(5)

,start,end
0,482209,2388000
1,3502663,3529630
2,4312416,4314288
3,6011424,6163959
4,6181414,6237353


In [166]:
comp_repeats_chr22.dropindex()

AttributeError: 'DataFrame' object has no attribute 'dropindex'

#### genes

In [139]:
genes= pd.read_csv('../genomic_data/t2t_annotation/chm13v2.0_RefSeq_Liftoff_v5.1.gff3' , sep='\t', comment='#', header=None)
genes.head(5)

,0,1,2,3,4,5,6,7,8
0,chr1,Liftoff,gene,6047,13941,.,-,.,ID=LOC124900618;gene_name=LOC124900618;db_xref...
1,chr1,Liftoff,transcript,6047,13941,.,-,.,ID=XR_002958507.2;Parent=LOC124900618;db_xref=...
2,chr1,Liftoff,exon,6047,6420,.,-,.,Parent=XR_002958507.2;db_xref=GeneID:124900618...
3,chr1,Liftoff,exon,12078,12982,.,-,.,Parent=XR_002958507.2;db_xref=GeneID:124900618...
4,chr1,Liftoff,exon,13445,13579,.,-,.,Parent=XR_002958507.2;db_xref=GeneID:124900618...


In [174]:
genes_chr22=genes[genes[0]=='chr22']
genes_chr22 = genes_chr22.drop([0,1,5,7,8], axis=1).reset_index(drop=True)
genes_chr22.columns=['type','start','end','strand']
genes_chr22.to_csv('../genomic_data/t2t_annotation/chr22/genes_chr22.csv', index=False)
genes_chr22

,type,start,end,strand
0,gene,11523,19018,+
1,transcript,11523,19018,+
2,exon,11523,11832,+
3,exon,15137,15415,+
4,exon,16907,19018,+
...,...,...,...,...
81181,CDS,51295685,51295791,-
81182,transcript,51297226,51313167,+
81183,exon,51297226,51297569,+
81184,exon,51298670,51298790,+


In [143]:
genes_chr22['type'].unique()

array(['gene', 'transcript', 'exon', 'CDS'], dtype=object)

#### repeat masker

In [6]:
RepeatMasker_path='../genomic_data/t2t_annotation/chm13v2.0_RepeatMasker_4.1.2p1.2022Apr14.bed'
RepeatMasker = pd.read_csv(RepeatMasker_path, sep='\t', header=None)
RepeatMasker

,0,1,2,3,4,5,6,7,8,9
0,chr1,41543,41632,MLT1K,263,+,LTR,ERVL-MaLR,27.9,4556095
1,chr1,85197,85334,L1M5,233,-,LINE,L1,25.7,4556096
2,chr1,152485,153701,teucerv1_5edge,10568,+,Unknown,undefined,0.4,4556097
3,chr1,153906,156321,teucerv2_3edge,8310,+,Unknown,undefined,18.6,4556101
4,chr1,159491,159734,teucerv2_3edge,490,+,Unknown,undefined,28.5,4556103
...,...,...,...,...,...,...,...,...,...,...
5590277,chrY,24586660,24586709,GAP_chr7_49bp,302,+,Unknown,undefined,10.2,4640159
5590278,chrY,26206468,26206513,GAP_chr7_49bp,260,+,Unknown,undefined,11.1,4640261
5590279,chrY,26206513,26206561,GAP_chr7_49bp,328,+,Unknown,undefined,6.2,4640262
5590280,chrY,26206561,26206610,GAP_chr7_49bp,309,+,Unknown,undefined,8.2,4640263


In [76]:
RepeatMasker_chr22=RepeatMasker[RepeatMasker[0]=='chr22']
RepeatMasker_chr22 = RepeatMasker_chr22.drop([0,4,7,8,9], axis=1).reset_index(drop=True)
RepeatMasker_chr22.columns=['start','end','repeat','strand','repeat_type']
RepeatMasker_chr22.sort_values(by='start')
RepeatMasker_chr22['length']=RepeatMasker_chr22['end']-RepeatMasker_chr22['start']

#RepeatMasker_chr22.to_csv('../genomic_data/t2t_annotation/chr22/RepeatMasker_chr22.csv', index=False)
RepeatMasker_chr22.sort_values(by='start')

,start,end,repeat,strand,repeat_type,length
1380,1,4578,(AACCCT)n,+,Simple_repeat,4577
1381,4578,4650,LTR60B,-,LTR,72
1382,4652,4809,LTR60B,-,LTR,157
1383,4815,5327,MER31-int,+,LTR,512
1384,5294,5511,MER31-int,+,LTR,217
...,...,...,...,...,...,...
90740,51319295,51319512,LTR60B,+,LTR,217
90741,51319512,51319817,AluYb8,+,SINE,305
90742,51319817,51320377,LTR60B,+,LTR,560
90743,51320377,51321954,TAR1,+,Satellite,1577


In [77]:
for x in RepeatMasker_chr22['repeat_type'].unique(): print(x)

LINE
LTR
Simple_repeat
Satellite
SINE
DNA
Low_complexity
srpRNA
RC
Unknown
tRNA
Retroposon
Beta
snRNA
scRNA
rRNA
DNA?


# Analyse repeats vs non-repeats sequences

### Analyse simple repeat sequences

In [7]:
def list_single_repeats(chr_list, repeats, min_repeat_nr=5):
    df = repeats[repeats[0].isin(chr_list)]
    df = df.drop([0, 4, 7, 8, 9], axis=1).reset_index(drop=True)
    df.columns = ['start', 'end', 'repeat', 'strand', 'repeat_type']
    df['length'] = df['end'] - df['start']
    df = df[df['repeat_type'] == 'Simple_repeat']
    df['single_repeat_length'] = df['repeat'].str.extract(r'\((.*?)\)n').applymap(len)
    df = df[df['length'] > min_repeat_nr * df['single_repeat_length']]
    df = df.groupby(['repeat', 'single_repeat_length'], as_index=False)['length'].sum()
    return df.sort_values(by='single_repeat_length')

In [8]:
known_repeats = list_single_repeats(['chr1'], RepeatMasker, min_repeat_nr=5)
known_repeats

/local/j262r/27972717/cluster_tmp/ipykernel_55702/4171794577.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df['single_repeat_length'] = df['repeat'].str.extract(r'\((.*?)\)n').applymap(len)


,repeat,single_repeat_length,length
0,(A)n,1,51345
1911,(G)n,1,1547
980,(C)n,1,2908
2626,(T)n,1,94294
1912,(GA)n,2,14172
...,...,...,...
780,(ATCATTGAATGGA)n,13,1394
2965,(TCATTGAATGGAA)n,13,1134
1945,(GAATGGAATCATT)n,13,82
2065,(GATGATTCCATTC)n,13,101


In [9]:
contamination = list_single_repeats(['chr21'], RepeatMasker, min_repeat_nr=5)
contamination

/local/j262r/27972717/cluster_tmp/ipykernel_55702/4171794577.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df['single_repeat_length'] = df['repeat'].str.extract(r'\((.*?)\)n').applymap(len)


,repeat,single_repeat_length,length
0,(A)n,1,7617
723,(G)n,1,210
958,(T)n,1,19080
411,(C)n,1,2249
129,(AC)n,2,34635
...,...,...,...
159,(ACATTGAAAT)n,10,60
293,(ATATATACAC)n,10,150
1257,(TTAATAGTAC)n,10,51
1107,(TCCATTCCAC)n,10,19699


In [10]:
unknown_repeats = list_single_repeats([f'chr{i}' for i in range (2,21)], RepeatMasker, min_repeat_nr=5)
unknown_repeats

/local/j262r/27972717/cluster_tmp/ipykernel_55702/4171794577.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df['single_repeat_length'] = df['repeat'].str.extract(r'\((.*?)\)n').applymap(len)


,repeat,single_repeat_length,length
0,(A)n,1,575592
8059,(T)n,1,1037087
3058,(C)n,1,26292
5728,(G)n,1,11817
1338,(AG)n,2,155405
...,...,...,...
690,(AATGGAATCATCG)n,13,88
9302,(TCGAATGGAATCA)n,13,131
2497,(ATCGAATGGAATC)n,13,616
2455,(ATCATCGAATGGA)n,13,148


In [11]:
def remove_self_multiples(df):
    # Function to check if a repeat string is made up of repeated subsequences
    def is_self_multiple(repeat_string):
        n = len(repeat_string)
        for i in range(1, n):
            if n % i == 0 and repeat_string[:i] * (n // i) == repeat_string:
                return True
        return False

    # Filter out rows where the repeat string is a self-multiple
    filtered_df = df[~df['repeat'].str.extract(r'\((.*?)\)n')[0].apply(is_self_multiple)]
    
    return filtered_df

In [12]:
known_repeats=remove_self_multiples(known_repeats)
unknown_repeats=remove_self_multiples(unknown_repeats)

In [13]:
known_repeat_strings = known_repeats['repeat'].str.extract(r'\((.*?)\)n')[0].tolist()
contamination_repeat_strings = contamination['repeat'].str.extract(r'\((.*?)\)n')[0].tolist()
unknown_repeats = unknown_repeats[~unknown_repeats['repeat'].str.extract(r'\((.*?)\)n')[0].isin(known_repeat_strings + contamination_repeat_strings)]
unknown_repeats

,repeat,single_repeat_length,length
4641,(CGA)n,3,278
9319,(TCGT)n,4,24
7322,(GGGC)n,4,588
9300,(TCGA)n,4,99
6666,(GCGA)n,4,1017
...,...,...,...
6959,(GGAATCATCGAAT)n,13,111
690,(AATGGAATCATCG)n,13,88
9302,(TCGAATGGAATCA)n,13,131
2455,(ATCATCGAATGGA)n,13,148


In [ ]:
import re

def simplify_repeat_strings(df):
    # Function to simplify the repeat string
    def simplify_repeat_string(repeat_string):
        match = re.match(r'\((.*?)\)n', repeat_string)
        if match:
            return match.group(1)
        return repeat_string

    df['repeat'] = df['repeat'].apply(simplify_repeat_string)
    return df

unknown_repeats = simplify_repeat_strings(unknown_repeats)
known_repeats = simplify_repeat_strings(known_repeats)

In [39]:
def expand_repeats_to_length(df, file_path, target_length=256):
    def repeat_to_length(subseq):
        return (subseq * (target_length // len(subseq) + 1))[:target_length]

    df['expanded_repeat'] = df['repeat'].apply(repeat_to_length)
    
    # Write the 'expanded_repeat' column to a text file
    df['expanded_repeat'].to_csv(file_path, index=False, header=False)
    return df
# Example usage
#known_repeats = expand_repeats_to_length(known_repeats)
#unknown_repeats = expand_repeats_to_length(unknown_repeats)
#unknown_repeats.head()

In [ ]:
known_repeats = expand_repeats_to_length(known_repeats, '../genomic_data/t2t_annotation/t2t_autoreg_analysis/known_expanded_repeats.txt')
unknown_repeats = expand_repeats_to_length(unknown_repeats, '../genomic_data/t2t_annotation/t2t_autoreg_analysis/unknown_expanded_repeats.txt')

In [9]:
import pickle

meta_path='../nanoGPT/data/genomic_char/architecture_nr_0_context_size_255/meta.pkl'
with open(meta_path, 'rb') as f:
    meta = pickle.load(f)
stoi, itos = meta['stoi'], meta['itos']
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [52]:
import numpy as np
with open('../genomic_data/t2t_annotation/t2t_autoreg_analysis/known_expanded_repeats.txt', 'r+') as f:
    context_size=255
    input_data = f.read().replace('\n', '')  # Remove newline characters
    num_samples = int(len(input_data) / (context_size + 1))
    print(num_samples)
    encoded_data = encode(input_data)
    #encoded_str = ''.join(map(str, encoded_data))  # Convert list of integers to a single string
    #f.seek(0)  # Move the file cursor to the beginning
    #f.write(encoded_str)
    #f.truncate()  # Truncate the file to the current position to remove any remaining old data
    print(encoded_data[:258])
    print(decode(encoded_data)[:258])
    known_ids = np.array(encoded_data, dtype=np.uint16)
    print(len(known_ids))

    known_ids.tofile('../genomic_data/t2t_annotation/t2t_autoreg_analysis/known_expanded_repeats.bin')


with open('../genomic_data/t2t_annotation/t2t_autoreg_analysis/unknown_expanded_repeats.txt', 'r+') as f:
    context_size=255
    input_data = f.read().replace('\n', '')  # Remove newline characters
    num_samples = int(len(input_data) / (context_size + 1))
    print(num_samples)
    encoded_data = encode(input_data)
    #encoded_str = ''.join(map(str, encoded_data))  # Convert list of integers to a single string
    #f.seek(0)  # Move the file cursor to the beginning
    #f.write(encoded_str)
    #f.truncate()  # Truncate the file to the current position to remove any remaining old data
    print(encoded_data[:258])
    print(decode(encoded_data)[:258])
    unknown_ids = np.array(encoded_data, dtype=np.uint16)
    #print(len(unknown_ids))
    unknown_ids.tofile('../genomic_data/t2t_annotation/t2t_autoreg_analysis/unknown_expanded_repeats.bin')

3713
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2]
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

In [50]:
known_repeats.to_csv('../genomic_data/t2t_annotation/t2t_autoreg_analysis/known_repeats.csv', index=False)
unknown_repeats.to_csv('../genomic_data/t2t_annotation/t2t_autoreg_analysis/unknown_repeats.csv', index=False)

In [7]:
# generate more random unknown repeats
import pandas as pd
import random
from tqdm import tqdm

# Function to read a chromosome file into a string
def read_chromosome(file_path):
    with open(file_path, 'r') as file:
        return file.read().upper().replace('\n', '')
chr1_content = read_chromosome('../genomic_data/t2t_genome/chr1.txt')

# Create the DataFrame
columns = ['single_repeat_length', 'repeat', 'expanded_repeat']
df = pd.DataFrame(columns=columns)

# Generate sequences for lengths from 10 to 127
all_sequences = []
for length in tqdm(range(10, 128), desc="Generating sequences"):
    for _ in range(99):
        seq =  ''.join(random.choices('ACGT', k=length))
        #print(seq, end=' ')
        expanded_seq = (seq * (256 // length + 1))[:256]
        if (seq * 2) not in chr1_content:
            all_sequences.append({'single_repeat_length': length, 'repeat': seq, 'expanded_repeat': expanded_seq})
        else: print(f'{seq*2} found in chr1')

df = pd.DataFrame(all_sequences, columns=columns)
print(df.head())

# Save the 'expanded_repeat' column to a text file
df['expanded_repeat'].to_csv('../genomic_data/t2t_annotation/t2t_autoreg_analysis/more_random_nonGenomic_repeats.txt', index=False, header=False)

# Save the entire DataFrame to a CSV file if needed
df.to_csv('../genomic_data/t2t_annotation/t2t_autoreg_analysis/more_random_nonGenomic_repeats.csv', index=False)


Generating sequences:   0%|                                                                                                                                                                                                                             | 0/118 [00:00<?, ?it/s]

AAAGACTTTTAAAGACTTTT found in chr1


Generating sequences: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 118/118 [1:23:45<00:00, 42.58s/it]


   single_repeat_length      repeat  \
0                    10  AACGTAAATT   
1                    10  GTAGGTCTAG   
2                    10  GAAAATGTCT   
3                    10  AGATGATTCG   
4                    10  ACCAAGCCAA   

                                     expanded_repeat  
0  AACGTAAATTAACGTAAATTAACGTAAATTAACGTAAATTAACGTA...  
1  GTAGGTCTAGGTAGGTCTAGGTAGGTCTAGGTAGGTCTAGGTAGGT...  
2  GAAAATGTCTGAAAATGTCTGAAAATGTCTGAAAATGTCTGAAAAT...  
3  AGATGATTCGAGATGATTCGAGATGATTCGAGATGATTCGAGATGA...  
4  ACCAAGCCAAACCAAGCCAAACCAAGCCAAACCAAGCCAAACCAAG...  


In [10]:
import numpy as np
sequences_file_path = '../genomic_data/t2t_annotation/t2t_autoreg_analysis/more_random_nonGenomic_repeats.txt'
# Read the sequences file and perform the encoding and saving steps
with open(sequences_file_path, 'r') as f:
    context_size = 255
    input_data = f.read().replace('\n', '')  # Remove newline characters
    num_samples = int(len(input_data) / (context_size + 1))
    print(num_samples)
    encoded_data = encode(input_data)
    print(encoded_data[:260])
    print(decode(encoded_data)[:260])
    known_ids = np.array(encoded_data, dtype=np.uint16)
    print(len(known_ids))
    known_ids.tofile('../genomic_data/t2t_annotation/t2t_autoreg_analysis/more_random_nonGenomic_repeats.bin')

11681
[0, 0, 1, 2, 3, 0, 0, 0, 3, 3, 0, 0, 1, 2, 3, 0, 0, 0, 3, 3, 0, 0, 1, 2, 3, 0, 0, 0, 3, 3, 0, 0, 1, 2, 3, 0, 0, 0, 3, 3, 0, 0, 1, 2, 3, 0, 0, 0, 3, 3, 0, 0, 1, 2, 3, 0, 0, 0, 3, 3, 0, 0, 1, 2, 3, 0, 0, 0, 3, 3, 0, 0, 1, 2, 3, 0, 0, 0, 3, 3, 0, 0, 1, 2, 3, 0, 0, 0, 3, 3, 0, 0, 1, 2, 3, 0, 0, 0, 3, 3, 0, 0, 1, 2, 3, 0, 0, 0, 3, 3, 0, 0, 1, 2, 3, 0, 0, 0, 3, 3, 0, 0, 1, 2, 3, 0, 0, 0, 3, 3, 0, 0, 1, 2, 3, 0, 0, 0, 3, 3, 0, 0, 1, 2, 3, 0, 0, 0, 3, 3, 0, 0, 1, 2, 3, 0, 0, 0, 3, 3, 0, 0, 1, 2, 3, 0, 0, 0, 3, 3, 0, 0, 1, 2, 3, 0, 0, 0, 3, 3, 0, 0, 1, 2, 3, 0, 0, 0, 3, 3, 0, 0, 1, 2, 3, 0, 0, 0, 3, 3, 0, 0, 1, 2, 3, 0, 0, 0, 3, 3, 0, 0, 1, 2, 3, 0, 0, 0, 3, 3, 0, 0, 1, 2, 3, 0, 0, 0, 3, 3, 0, 0, 1, 2, 3, 0, 0, 0, 3, 3, 0, 0, 1, 2, 3, 0, 0, 0, 3, 3, 0, 0, 1, 2, 3, 0, 2, 3, 0, 2]
AACGTAAATTAACGTAAATTAACGTAAATTAACGTAAATTAACGTAAATTAACGTAAATTAACGTAAATTAACGTAAATTAACGTAAATTAACGTAAATTAACGTAAATTAACGTAAATTAACGTAAATTAACGTAAATTAACGTAAATTAACGTAAATTAACGTAAATTAACGTAAATTAACGTAAATTAACGTAAATTAACGTAAATTAAC

### Analyse non-repeat sequences

In [16]:
import pandas as pd
import random
from tqdm import tqdm

# Paths to the RepeatMasker file and chromosome files
RepeatMasker_path = '../genomic_data/t2t_annotation/chm13v2.0_RepeatMasker_4.1.2p1.2022Apr14.bed'
chr1_path = '../genomic_data/t2t_genome/chr1.txt'
chr2_path = '../genomic_data/t2t_genome/chr2.txt'

# Read the RepeatMasker file
RepeatMasker = pd.read_csv(RepeatMasker_path, sep='\t', header=None)

# Get the largest index of chr1
max_chr1_index = RepeatMasker[RepeatMasker[0] == 'chr1'][2].max()

# Function to generate non-repeating indices for a chromosome
def generate_non_repeating_indices(repeat_masker, chromosome, max_index, num_indices=500, sequence_length=256):
    indices = []
    progress = tqdm(total=num_indices, desc=f"Generating non-repeating indices for {chromosome}")
    while len(indices) < num_indices:
        idx = random.randint(0, max_index - sequence_length)
        overlapping = repeat_masker[(repeat_masker[0] == chromosome) & (repeat_masker[1] <= idx) & (repeat_masker[2] >= idx)]
        if overlapping.empty:
            indices.append(idx)
            progress.update(1)
    progress.close()
    return indices

# Generate non-repeating indices for chr1
chr1_indices = generate_non_repeating_indices(RepeatMasker, 'chr1', max_chr1_index)

# Function to read a chromosome file into a string
def read_chromosome(file_path):
    with open(file_path, 'r') as file:
        return file.read().upper()

# Read the content of chr1 and chr2
chr1_content = read_chromosome(chr1_path)
chr2_content = read_chromosome(chr2_path)

# Extract sequences from chr1 ensuring they occur exactly once
chr1_sequences = []
unique_chr1_indices = []
print("Extracting sequences from chr1...")
for idx in tqdm(chr1_indices, desc="Processing chr1 sequences"):
    seq = chr1_content[idx:idx+256]
    if chr1_content.count(seq) == 1:
        chr1_sequences.append(seq)
        unique_chr1_indices.append(idx)
        #print(f"Known sequence added: {seq[:10]}... at index {idx}")
        if len(chr1_sequences) >= 500:
            break

# Check for uniqueness in chr2 and collect sequences
unique_sequences = []
labels = []

for seq in chr1_sequences:
    unique_sequences.append(seq)
    labels.append('known')

# Get the largest index of chr2
max_chr2_index = RepeatMasker[RepeatMasker[0] == 'chr2'][2].max()

# Generate non-repeating indices for chr2
chr2_indices = generate_non_repeating_indices(RepeatMasker, 'chr2', max_chr2_index)

print("Extracting sequences from chr2 and checking against chr1...")
for idx in tqdm(chr2_indices, desc="Processing chr2 sequences"):
    seq = chr2_content[idx:idx+256]
    if seq not in chr1_content:
        unique_sequences.append(seq)
        labels.append('unknown')
        #print(f"Unknown sequence added: {seq[:10]}... at index {idx}")
        if len([label for label in labels if label == 'unknown']) >= 500:
            break

# Create the DataFrame
df = pd.DataFrame({
    'sequence': unique_sequences,
    'label': labels
})

# Display the first few rows of the DataFrame
print(df.head())

# Save the DataFrame to a file if needed
df.to_csv('../genomic_data/t2t_annotation/t2t_autoreg_analysis/non_repeats.csv', index=False)



Generating non-repeating indices for chr1: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [08:16<00:00,  1.01it/s]


Extracting sequences from chr1...



Processing chr1 sequences: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [04:26<00:00,  1.88it/s]

Generating non-repeating indices for chr2: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [07:56<00:00,  1.05it/s]


Extracting sequences from chr2 and checking against chr1...



Processing chr2 sequences: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [04:18<00:00,  1.93it/s]

                                            sequence  label
0  TTCAATGGACACATCTGTATTTGCAGAAAATGATGAAGATGAAGAT...  known
1  TTAAACATGTAGACAAAGAATTATTTTATGTTTAATATGTCCACCA...  known
2  GGTTCTATGAAGCTGCTTCGGCCATTAGTAGAGAAAATGAAGTTTC...  known
3  CCTAATTATCAATTGTTTCATATCACGTGAGAATGCTGAAAATCAA...  known
4  GAACAGATGAGCTGGAGGCTGCAATGATCAGAACATTAGCTAGAGG...  known


In [19]:
import numpy as np
# Save sequences to a text file
sequences_file_path = '../genomic_data/t2t_annotation/t2t_autoreg_analysis/non_repeats.txt'
with open(sequences_file_path, 'w') as f:
    for seq in df['sequence']:
        f.write(f"{seq}\n")
# Read the sequences file and perform the encoding and saving steps
with open(sequences_file_path, 'r') as f:
    context_size = 255
    input_data = f.read().replace('\n', '')  # Remove newline characters
    num_samples = int(len(input_data) / (context_size + 1))
    print(num_samples)
    encoded_data = encode(input_data)
    print(encoded_data[:258])
    print(decode(encoded_data)[:258])
    known_ids = np.array(encoded_data, dtype=np.uint16)
    print(len(known_ids))
    known_ids.tofile('../genomic_data/t2t_annotation/t2t_autoreg_analysis/non_repeats.bin')


991
[3, 3, 1, 0, 0, 3, 2, 2, 0, 1, 0, 1, 0, 3, 1, 3, 2, 3, 0, 3, 3, 3, 2, 1, 0, 2, 0, 0, 0, 0, 3, 2, 0, 3, 2, 0, 0, 2, 0, 3, 2, 0, 0, 2, 0, 3, 2, 0, 2, 2, 0, 3, 2, 0, 0, 2, 0, 1, 2, 0, 1, 0, 0, 0, 2, 0, 1, 2, 0, 2, 2, 0, 2, 2, 3, 3, 2, 0, 2, 0, 0, 0, 2, 3, 0, 1, 0, 2, 2, 0, 0, 3, 1, 0, 1, 1, 3, 2, 1, 1, 1, 1, 1, 0, 2, 2, 3, 0, 0, 1, 0, 3, 3, 2, 0, 0, 3, 0, 0, 3, 1, 0, 2, 2, 0, 2, 1, 2, 2, 2, 3, 0, 0, 3, 2, 2, 2, 3, 2, 2, 3, 0, 0, 0, 0, 3, 0, 3, 2, 0, 0, 0, 0, 0, 2, 2, 3, 1, 3, 1, 0, 2, 0, 0, 0, 2, 0, 0, 3, 0, 0, 0, 0, 2, 2, 2, 0, 2, 2, 3, 2, 0, 0, 2, 2, 3, 0, 2, 3, 1, 0, 1, 0, 2, 0, 3, 3, 1, 1, 0, 2, 0, 2, 2, 1, 0, 2, 2, 0, 3, 0, 0, 0, 2, 0, 0, 0, 2, 1, 3, 2, 1, 0, 2, 0, 2, 3, 2, 1, 0, 1, 3, 2, 0, 3, 3, 3, 1, 0, 3, 2, 3, 2, 1, 3, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 2, 3, 3]
TTCAATGGACACATCTGTATTTGCAGAAAATGATGAAGATGAAGATGAGGATGAAGACGACAAAGACGAGGAGGTTGAGAAAGTACAGGAATCACCTGCCCCCAGGTAACATTGAATAATCAGGAGCGGGTAATGGGTGGTAAAATATGAAAAAGGTCTCAGAAAGAATAAAAGGGAGGTGAAGGTAGTCACAGATTCCAGAGGCAGGATAAAGAAAGCTG